In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NHITS

# Fetch data from the API for training (1 year) and testing (1 month)
url_train = "https://api.energy-charts.info/price?bzn=DE-LU&start=2024-01-01&end=2024-09-01"
url_test = "https://api.energy-charts.info/price?bzn=DE-LU&start=2024-09-01&end=2024-10-01"

response_train = requests.get(url_train)
response_test = requests.get(url_test)

data_train = response_train.json()
data_test = response_test.json()

# Convert to DataFrame
train_df = pd.DataFrame({
    'timestamp': pd.to_datetime(data_train['unix_seconds'], unit='s'),
    'price': data_train['price']
})
test_df = pd.DataFrame({
    'timestamp': pd.to_datetime(data_test['unix_seconds'], unit='s'),
    'price': data_test['price']
})

# Clean the data (remove rows with null prices)
train_df = train_df.dropna()
test_df = test_df.dropna()

# Rename columns for modeling
train_df = train_df.rename(columns={'timestamp': 'ds', 'price': 'y'})
test_df = test_df.rename(columns={'timestamp': 'ds', 'price': 'y'})
train_df['unique_id'] = 'electricity_prices'
test_df['unique_id'] = 'electricity_prices'



In [ ]:

# Define models with a 1-day forecast horizon
models = [
    LSTM(h=1, max_steps=200, scaler_type='standard', encoder_hidden_size=64, decoder_hidden_size=64),
    NHITS(h=1, input_size=2, max_steps=100, n_freq_downsample=[2, 1, 1])
]

# Initialize the NeuralForecast object
nf = NeuralForecast(models=models, freq='H')
nf.fit(df=train_df)

# Prepare for iterative day-ahead forecasting
forecasted_values = []  # List to store the day-by-day forecasts
input_df = train_df.copy()  # Start with the training data as the initial input

# Perform day-ahead predictions for each day in the test set
for i in range(len(test_df)):
    # Generate the forecast for the next day
    day_forecast = nf.predict().iloc[-1:]  # Take the last forecasted day
    
    # Append the forecast to the list for later use in plotting
    forecasted_values.append({
        'ds': test_df.iloc[i]['ds'],
        'LSTM': day_forecast['LSTM'].values[0],
        'NHITS': day_forecast['NHITS'].values[0]
    })
    
    # Update the input data with the actual value of the next day in the test set
    next_row = {'ds': test_df.iloc[i]['ds'], 'y': test_df.iloc[i]['y'], 'unique_id': 'electricity_prices'}
    input_df = pd.concat([input_df, pd.DataFrame([next_row])])
    nf.fit(df=input_df)  # Re-train or update the model with the expanded dataset

# Convert the forecast list to a DataFrame for plotting
forecast_df = pd.DataFrame(forecasted_values)

# Ensure 'ds' columns are in datetime format
test_df['ds'] = pd.to_datetime(test_df['ds'])
forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])



Epoch 83:   0%|          | 0/1 [00:00<?, ?it/s, v_num=65, train_loss_step=0.168, train_loss_epoch=0.168]        

In [ ]:
# Plotting
fig, ax = plt.subplots(1, 1, figsize=(20, 7))
test_df.set_index('ds')['y'].plot(ax=ax, linewidth=2, label='Actual Price')
forecast_df.set_index('ds')['LSTM'].plot(ax=ax, linewidth=2, label='LSTM Prediction')
forecast_df.set_index('ds')['NHITS'].plot(ax=ax, linewidth=2, label='NHITS Prediction')

ax.set_title('Electricity Price Forecast (Day-by-Day)', fontsize=22)
ax.set_ylabel('Price', fontsize=20)
ax.set_xlabel('Date', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()
plt.show()